# Set Membership

The cell below defines two **abstract classes**: the first represents a set and basic insert/search operations on it. You will need to impement this API four times, to implement (1) sequential search, (2) binary search tree, (3) balanced search tree, and (4) bloom filter. The second defines the synthetic data generator you will need to implement as part of your experimental framework. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [192]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

# abstract class to represent a set and its insert/search operations
class AbstractSet(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # inserts "element" in the set
    # returns "True" after successful insertion, "False" if the element is already in the set
    # element : str
    # inserted : bool
    @abstractmethod
    def insertElement(self, element):     
        inserted = False
        return inserted   
    
    # checks whether "element" is in the set
    # returns "True" if it is, "False" otherwise
    # element : str
    # found : bool
    @abstractmethod
    def searchElement(self, element):
        found = False
        return found    
    
    
    
# abstract class to represent a synthetic data generator
class AbstractTestDataGenerator(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # creates and returns a list of length "size" of strings
    # size : int
    # data : list<str>
    @abstractmethod
    def generateData(self, size):     
        data = [""]*size
        return data   


Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [193]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE
class Node:
    def __init__(self, val):
        self.val = val
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def show(self):
        current = self.head
        while(current is not None):
            print(current.val)
            current = current.next    

Use the cell below to implement the requested API by means of **sequential search**.

In [194]:
class SequentialSearchSet(AbstractSet):
    
    def __init__(self):   
        self.set = LinkedList()
        
    def insertElement(self, element):
        inserted = False
        current = self.set.head

        # case 1: insert to empty linkedlist
        if current is None:
            self.set.head = Node(element)
        # case 2: insert to head of linkedlist
        elif current.val > element:
            newNode = Node(element)
            newNode.next = self.set.head
            self.set.head = newNode
        # case 3: insert to somewhere else in the linkedlist
        else:
            while(current.next is not None and current.next.val <= element):
                current = current.next
            if current.val == element:
                return inserted
            else:
                newNode = Node(element)
                newNode.next = current.next
                current.next = newNode

        inserted = True
        return inserted


    def searchElement(self, element):     
        found = False
        if self.set is None:
            return found
        else:
            current = self.set.head
            while(current is not None):
                if current.val == element:
                    found = True
                    return found
                current = current.next

        return found    

Dash: just for testing so i'll remove these cells later

In [ ]:
f = open("test2-warpeace.txt", "r")
test = SequentialSearchSet()
for line in f:
    for word in line.split():
        test.insertElement(word)
test.set.show()
f.close()

In [ ]:
f = open("test-search.txt", "r")
for word in f:
    target = word.strip()
    if(test.searchElement(target)):
        print(target + " ✓")
    else:
        print(target + " ✕")

Use the cell below to implement the requested API by means of **binary search tree**.

In [4]:
class BinarySearchTreeSet(AbstractSet):
    
    def __init__(self):
        # ADD YOUR CODE HERE

        
        pass           
     
    
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found    

Use the cell below to implement the requested API by means of **balanced search tree**.

In [5]:
class BalancedSearchTreeSet(AbstractSet):
    
    def __init__(self):
        # ADD YOUR CODE HERE

        
        pass           
     
    
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found    

Use the cell below to implement the requested API by means of **bloom filter**.

In [6]:
from bitarray import bitarray

# calculate natural logarithm
def ln(x):
    n = 1000.0
    return n * ((x ** (1/n)) - 1)

# get size of bitarray
def get_size(num_of_items, fp_prob):
        size = -(num_of_items * ln(fp_prob)) / (ln(2) ** 2)
        return int(size)

# get number of hash function to be used
def get_hash_num(bitarray_size, num_of_items):
    hash_num = (bitarray_size / num_of_items) * ln(2)
    return int(hash_num)

# Murmurhash3 32bits version
# reference: https://huagetai.github.io/posts/fcfde8ff/
def murmurhash_32(key, seed=0, blocksize=64):
    h = seed
    constant1 = 0xcc9e2d51
    constant2 = 0x1b873593
    r1 = 15
    r2 = 13
    m = 5
    n = 0xe6546b64
    bits_32 = 0xffffffff
    # Divide key into chunks of blocksize bits (default 64 bits) and iterate over each chunk
    for chunk in [key[i:i+blocksize//8] for i in range(0, len(key), blocksize//8)]:
        k = 0
        for i, c in enumerate(chunk):
            # Convert chunk into integer k by shifting bytes left by multiples of 8 bits
            k |= ord(c) << (8 * i)
        k *= constant1
        k &= bits_32  # Truncate k to 32 bits
        k = (k << r1) | (k >> 32 - r1)  # Rotate k left by 15 bits and bitwise OR with k rotated right by 17 bits
        k *= constant2
        k &= bits_32
        h ^= k  # XOR
        h = (h << r2) | (h >> 32 - r2)
        h = h * m + n
        h &= bits_32
    h ^= len(key)
    h &= bits_32
    h ^= h >> 16
    h *= 0x85ebca6b
    h &= bits_32
    h ^= h >> 13
    h *= 0xc2b2ae35
    h &= bits_32
    h ^= h >> 16
    h &= bits_32
    return h



class BloomFilterSet(AbstractSet):
    
    def __init__(self, num_of_items, fp_prob = 0.05):
        # ADD YOUR CODE HERE
        self.size = get_size(num_of_items, fp_prob)
        self.hash_num = get_hash_num(self.size, num_of_items)
        self.bit_array = bitarray(self.size)
        self.bit_array.setall(0)




    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
        for i in range(self.hash_num):
            val_of_hash = murmurhash_32(element, i) % self.size
            if self.bit_array[val_of_hash] == 0:
                inserted = True
                self.bit_array[val_of_hash] = 1
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE
        for i in range(self.hash_num):
            val_of_hash = murmurhash_32(element, i) % self.size
            if self.bit_array[val_of_hash] == 0:
                return found
        found = True
        return found

In [ ]:
import random
word_present = ['abound','abounds','abundance','abundant','accessible',
                'bloom','blossom','bolster','bonny','bonus','bonuses',
                'coherent','cohesive','colorful','comely','comfort',
                'gems','generosity','generous','generously','genial']

word_absent = ['bluff','cheater','hate','war','humanity',
               'racism','hurt','nuke','gloomy','facebook',
               'geeksforgeeks','twitter']

bloom_filter = BloomFilterSet(len(word_present))
for word in word_present:
    bloom_filter.insertElement(word)
random.shuffle(word_present)
random.shuffle(word_absent)
test_words = word_present[:10] + word_absent
random.shuffle(test_words)
for word in test_words:
    if bloom_filter.searchElement(word):
        if word in word_absent:
            print("'{}' is a false positive!".format(word))
        else:
            print("'{}' is probably present!".format(word))
    else:
        print("'{}' is definitely not present!".format(word))

Use the cell below to implement the **synthetic data generator** as part of your experimental framework.

In [7]:
import string
import random

class TestDataGenerator(AbstractTestDataGenerator):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        
        
        pass           
        
    def generateData(self, size):     
        # ADD YOUR CODE HERE
        data = [""]*size
        

        return data   



Use the cells below for the python code needed to **fully evaluate your implementations**, first on real data and subsequently on synthetic data (i.e., read data from test files / generate synthetic one, instantiate each of the 4 set implementations in turn, then thorouhgly experiment with insert/search operations and measure their performance).

In [8]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON REAL DATA





In [9]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON SYNTHETIC DATA



